<a href="https://colab.research.google.com/github/asigalov61/Incredible-REMI-Piano/blob/master/Incredible_REMI_Piano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Try REMI Pop Music Transformer yourself with this simple Google Colab

In [ ]:
%tensorflow_version 1.x
!pip install gast==0.2.2
!pip install tensorflow-gpu==1.14.0
!pip install keras
!pip install miditoolkit
!git clone https://github.com/asigalov61/remi.git

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.chdir('remi')
from model import PopMusicTransformer

In [ ]:
!wget 'https://incredibleremipiano.s3-us-west-1.amazonaws.com/REMI-tempo-checkpoint.zip'
!unzip 'REMI-tempo-checkpoint.zip'

In [ ]:
model = PopMusicTransformer(checkpoint='REMI-tempo-checkpoint', is_training=False)

In [4]:
    # generate from scratch
    model.generate(
        n_target_bar=16,
        temperature=1.2,
        topk=5,
        output_path='./result/from_scratch.midi',
        prompt=None)

In [ ]:
    # generate continuation
    model.generate(
        n_target_bar=16,
        temperature=1.2,
        topk=5,
        output_path='./result/continuation.midi',
        prompt='/content/remi/prompt1.mid')

In [ ]:
# close model
model.close()